In [1]:
import sys
sys.path.append('D:\pysource\machine-study')

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [3]:
num_hiddens=256
rnn_layer=nn.RNN(len(vocab), num_hiddens)

In [4]:
state=torch.zeros((1,batch_size,num_hiddens))
state.shape  #隐藏层数 批量大小 隐藏层

torch.Size([1, 32, 256])

In [5]:
x=torch.rand(size=(num_steps,batch_size,len(vocab)))
y,state_new=rnn_layer(x,state)
#步长 批量大小，隐藏层
y.shape,state_new.shape


(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [6]:
class RNNModel(nn.Module):
    def __init__(self, num_hiddens, vocab_size):
        super().__init__()
        self.rnn=nn.RNN(vocab_size,num_hiddens)
        self.vocab_size=vocab_size
        self.hidden_nums=self.rnn.hidden_size
        self.linear=nn.Linear(self.hidden_nums,self.vocab_size)

    def forward(self,inputs,state):
        x=F.one_hot(inputs.T,self.vocab_size).to(torch.float32) #35*32*28 32*255
        y,state=self.rnn(x,state)
        #outputs=self.linear(y)
        #输出形状是步长*批量大小，隐藏层数
        outputs=self.linear(y.reshape((-1,y.shape[-1])))
        return outputs,state

    def begin_state(self,batch_size):
        return torch.zeros((self.rnn.num_layers,batch_size,self.hidden_nums))
    

# trnn=RNNModel(rnn_layer,len(vocab)) 
# state=trnn.begin_state(32)
# x,y=next(iter(train_iter))
# y,state=trnn(x,state)



In [7]:
#预测数据，把数据当成批量1 的数据处理，用输出的最后一个字母，作为输入进行预测
def pred_ch8(prefix,pred_num,net,vocab):
    state=net.begin_state(1)
    outputs=[vocab[prefix[0]]]
    get_input=lambda : torch.tensor([outputs[-1]]).reshape((1,1))
    for i in prefix[1:]:  #预热预测数据
        _,state=net(get_input(),state)
        outputs.append(vocab[i])
    for _ in range(pred_num):
        y,state=net(get_input(),state)
        outputs.append(int(y.argmax(dim=1)))

    return ''.join([ vocab.idx_to_token[out] for out in outputs])

test=RNNModel(num_hiddens,len(vocab))
pred_ch8("time traveller",10,test,vocab)


'time travellerwwwwwwwwww'

In [ ]:
def grad_clipping(net,theta):
    # params=net.parameters()
    # norm=torch.sqrt(sum(torch.sum(p.grad**2) for p in params))
    params=[p for p in net.parameters() if p.requires_grad]
    norm=torch.sqrt(sum(torch.sum(p.grad**2) for p in params))
    # print(norm_k,' ',norm)
    if norm>theta:
        for p in params:
            p.grad[:]*=theta/norm


def train_ch8(net,loss,updater,train_iter):
    state=None
    ct=0
    for X,Y in train_iter:
        if state==None:
            state=net.begin_state(X.shape[0])
        else:
            state.detach_()
        
        y=Y.T.reshape(-1)
        y_hat,state=net(X,state)
        l=loss(y_hat,y.long()).mean()
        updater.zero_grad()
        l.backward()
        grad_clipping(net,1)
        updater.step()
        ct=ct+1
    return ct
        

def train_rnn(epoch,net,lr,train_iter,vocab):
    loss=nn.CrossEntropyLoss()
    updater=torch.optim.SGD(net.parameters(),lr)
    
    predict=lambda prefix:pred_ch8(prefix,50,net,vocab)
    count=0
    for i in range(epoch):
        count+=train_ch8(net,loss,updater,train_iter);
    print(count)
    print(predict("time traveller"))
    print(predict("traveller"))

epoch=512
lr=1
num_hiddens=256
net=RNNModel(num_hiddens,len(vocab))
train_rnn(epoch,net,lr,train_iter,vocab)

4000
time traveller after the pauserequired for the ableplyound for a
traveller poon the milaclwtoments in time tare dinett anopt
